# Custom substructure loading from PDB and NAGL charge assignment

This is a RED example showcasing early prototypes of functionality that we plan to make available in the coming year. We provide no guarantee of stability or correct operation, but this is an example of what may be to come.

This example will:
* Load a modified protein from PDB using `Topology.from_pdb`
* Create a librarycharge for just the modified residue and create the OpenMM system with ff14SB parameters

See also:
https://gist.github.com/Yoshanuikabundi/66007cb9966b1455a259baaf7cd7e7c3#file-ncaa_parametrization-ipynb

Why it's red:
- NAGL is experimental and pre-release
- Charging a whole protein with NAGL will likely not give the same charges as FF14SB
- Sage parameters have not been rigourously validated for use in non-canonical amino acids.

## Introduction

Here we have a PDB structure containing a protein with sequence "ACE-ALA5-DYE-ALA5-NME". The DYE residue is a cysteine labelled with a fluorophore via a maleimide "click" reaction.

In [ ]:
import mdtraj
import nglview
import numpy as np
import openff.toolkit.utils.exceptions
import openmm
import openmm.unit as omm_unit
import rdkit.Chem
from chemper.graphs.single_graph import SingleGraph
from chemper.mol_toolkits import mol_toolkit
from openff.toolkit import ForceField, Molecule
from openff.toolkit.typing.engines.smirnoff.parameters import LibraryChargeHandler
from openff.units import unit
from openff.units.openmm import from_openmm
from openmm.app import DCDReporter
from pdbfixer import PDBFixer

In [ ]:
import nglview

w = nglview.show_file("inputs/ace-a5-dye-a5-nme_solvated.pdb")
w.clear_representations()
w.add_representation("cartoon")
w.add_representation("licorice", selection="not (HOH NA CL)")
w.add_representation("spacefill", selection="NA CL")
w.add_representation("spacefill", selection="HOH", opacity=0.05)
w

## Load a PDB with non-canonical amino acid


We're using OpenMM's PDBFile class under the hood for the initial loading. This leads to the following requirements in the input PDB:

* Standard residues must have canonical residue and atom names
* Modified residues must have elements explicitly listed in the last column
* The connectivity (but not bond order) of the modified residue must be explicitly specified using CONECT records (including peptide bonds to adjacent residues)

Let's get started on the workflow by making a representation of the non-canonical amino acid, which we'll use to teach our PDB loader about the new substructure. I'm labeling the connection points with [Fr] atoms.

In [ ]:
from openff.toolkit import Molecule, Topology

# Make unnatural AA as its own mol. Since a residue is a substructure, not a valid molecule,
# use something that's easy to recognize as a "cap". Here I use Fr since it makes one bond,
# we're super unlikely to see Fr anywhere else, and Sage doesn't support it, so if
# we make a mistake and it gets left behind we'll eventually get an error.
substructure_mol = Molecule.from_smiles(
    "C1=CC2=C(C=C1N3C(=O)[C@@H](SC[C@H](C(=O)[Fr])N[Fr])CC3=O)C(=O)O[C@]24C5=C(C=C(C=C5)O)OC6=C4C=CC(=C6)O"
)
substructure_mol

In [ ]:
# Label the atoms with whether they're leaving
for atom in substructure_mol.atoms:
    if atom.symbol == "Fr":
        atom.metadata["substructure_atom"] = False
    else:
        atom.metadata["substructure_atom"] = True

In [ ]:
top = Topology.from_pdb(
    "inputs/ace-a5-dye-a5-nme_solvated.pdb",
    # _additional_substructures is a PROTOTYPE.
    # Its behavior and input type are likely to change.
    _additional_substructures=[substructure_mol],
)

# And it's loaded!
peptide = top.molecule(0)
peptide.visualize(backend="rdkit")

In [ ]:
w = top.visualize()
w.clear_representations()
w.add_representation("cartoon")
w.add_representation("licorice", selection="not (HOH NA CL)")
w.add_representation("spacefill", selection="NA CL")
w.add_representation("spacefill", selection="HOH", opacity=0.05)
w

## Compute NAGL partial charges

Here, for simplicity, we simply compute NAGL charges for the entire peptide. A more rigorous approach might combine NAGL or Antechamber charges for the dye with Amber library charges for the other residues, as was done in a previous workshop:

https://www.youtube.com/watch?v=lZ4UUgHQpWg

In [ ]:
from openff.nagl_models import list_available_nagl_models

[path.name for path in list_available_nagl_models()]

In [ ]:
from openff.toolkit.utils.nagl_wrapper import NAGLToolkitWrapper

# Do a "sacrificial" first run of nagl on methane so the model is compiled in this cell
Molecule.from_smiles("C").assign_partial_charges(
    "openff-gnn-am1bcc-0.1.0-rc.1.pt", toolkit_registry=NAGLToolkitWrapper()
)

In [ ]:
%%time
peptide.assign_partial_charges(
    "openff-gnn-am1bcc-0.1.0-rc.1.pt", toolkit_registry=NAGLToolkitWrapper()
)
print(peptide.partial_charges)

## Parametrize and simulate the peptide with Sage, Amber ff14sb, and NAGL charges

In [ ]:
sage_ff14sb = ForceField("openff-2.1.1.offxml", "ff14sb_off_impropers_0.0.3.offxml")

In [ ]:
ic = sage_ff14sb.create_interchange(
    top, 
    charge_from_molecules=[peptide]
)

In [ ]:
import openmm
from openff.units import Quantity, unit

simulation = ic.to_openmm_simulation(
    openmm.LangevinIntegrator(
        300 * openmm.unit.kelvin,
        1 / openmm.unit.picosecond,
        2 * openmm.unit.femtoseconds,
    )
)

# Energy minimize
simulation.minimizeEnergy()

# Add a reporter to record the structure every 1000 steps
dcd_reporter = openmm.app.DCDReporter("ncaa.dcd", 1000)
simulation.reporters.append(dcd_reporter)
simulation.context.setVelocitiesToTemperature(simulation.integrator.getTemperature())

In [ ]:
simulation.runForClockTime(1 * openmm.unit.minute)

In [ ]:
from utils import nglview_show_openmm

w = nglview_show_openmm(simulation.topology, "ncaa.dcd")
w.clear_representations()
w.add_representation("cartoon")
w.add_representation("licorice", selection="not (HOH NA CL)")
w.add_representation("spacefill", selection="NA CL")
w.add_representation("spacefill", selection="HOH", opacity=0.05)
w